In [ ]:
import pandas as pd
import datetime 
dfMatches = pd.read_csv("matchInfos.csv")
dfMatches.loc["date"] = pd.to_datetime(dfMatches["date"])


In [6]:
# only select 14. Spieltag (for testing)
df_try = dfMatches[dfMatches["matchday"] == "14. Spieltag"]

In [ ]:


HEADERS = {
    "DB-Client-Id": DB_CLIENT_ID,
    "DB-Api-Key": DB_API_KEY
}



In [55]:
import requests
import xml.etree.ElementTree as ET

# Konfiguration
EVA_ORIGIN = "8011160"  # Berlin Hbf
DATE = "251110"         # YYMMDD
HOUR = "08"             # Stunde
DESTINATION = "Hamburg Hbf"

HEADERS = {
    "Accept": "application/xml",
    "DB-Client-Id": "dein_client_id",
    "DB-Api-Key": "dein_api_key"
}

url = f"https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/plan/{EVA_ORIGIN}/{DATE}/{HOUR}"

response = requests.get(url, headers=HEADERS)

print("Statuscode:", response.status_code)
if response.status_code != 200:
    print("Fehler:", response.text)
else:
    root = ET.fromstring(response.content)
    # Filter nach Ziel
    for departure in root.findall(".//Departure"):
        direction = departure.find("direction").text
        if direction == DESTINATION:
            train = departure.find("train").text
            time = departure.find("time").text
            print(f"Zug {train} nach {direction} ab {time}")



Statuscode: 401
Fehler: <?xml version="1.0" encoding="UTF-8"?>
<errorResponse>
  <httpCode>401</httpCode>
  <httpMessage>Unauthorized</httpMessage>
  <moreInformation>Invalid client id or secret.</moreInformation>
</errorResponse>


In [40]:
import requests

HEADERS = {
    "DB-Client-Id": DB_CLIENT_ID,
    "DB-Api-Key": DB_API_KEY,
    "Accept": "application/xml"
}

def fetch_raw_route(eva_from: str, eva_to: str, date="2025-11-06T08:00"):
    url = "https://apis.deutschebahn.com/db-api-marketplace/apis/fahrplan/v1/connections"
    params = {
        "from": eva_from,
        "to": eva_to,
        "date": date
    }
    
    response = requests.get(url, headers=HEADERS, params=params)
    
    if response.status_code != 200:
        raise Exception(f"Fehler: {response.status_code} | {response.text}")
    
    # Hier einfach die rohe XML ausgeben
    print(response.text)
    return response.text

fetch_raw_route("8098160", "8011160")


Exception: Fehler: 404 | <?xml version="1.0" encoding="UTF-8"?>
<errorResponse>
  <httpCode>404</httpCode>
  <httpMessage>Not Found</httpMessage>
  <moreInformation>API not found for requested URI</moreInformation>
</errorResponse>

In [18]:
df_try.head()

,match_id,league,date,matchday,team_home,team_away,stadium,capacity,home_city,away_city,home_station,home_lat,home_lon,away_station,away_lat,away_lon
117,77379,bl1,2025-12-12 20:30:00,14. Spieltag,1. FC Union Berlin,RB Leipzig,Stadion An der Alten Försterei,22012,Berlin,Leipzig,Berlin Hauptbahnhof,52.524945,13.369661,Leipzig Hauptbahnhof,51.346552,12.383086
118,77375,bl1,2025-12-13 15:30:00,14. Spieltag,Eintracht Frankfurt,FC Augsburg,Deutsche Bank Park,59500,Frankfurt (Main),Augsburg,Frankfurt (Main) Hauptbahnhof,50.106654,8.662581,Augsburg Hauptbahnhof,48.365595,10.886355
119,77378,bl1,2025-12-13 15:30:00,14. Spieltag,Borussia Mönchengladbach,VfL Wolfsburg,Stadion im Borussia-Park,54042,Mönchengladbach,Wolfsburg,Mönchengladbach Hauptbahnhof,51.16322,6.439669,Wolfsburg Hauptbahnhof,52.429161,10.787807
120,77380,bl1,2025-12-13 15:30:00,14. Spieltag,FC St. Pauli,1. FC Heidenheim 1846,Millerntor-Stadion,29546,Hamburg,Heidenheim,Hamburg Hauptbahnhof,53.553199,10.006436,Heidenheim (Zentrum),48.676764,10.152923
121,77381,bl1,2025-12-13 15:30:00,14. Spieltag,TSG Hoffenheim,Hamburger SV,PreZero Arena,30150,Sinsheim,Hamburg,Sinsheim Hauptbahnhof,49.250168,8.8755,Hamburg Hauptbahnhof,53.553199,10.006436


In [23]:
example_match = df_try.iloc[0]  # erstes Spiel
game_start = example_match["date"]

# Anreise: 3–6 Stunden vorher
departure_time = game_start - timedelta(hours=3)

connections = get_db_connections(
    from_station=example_match["away_station"],
    to_station=example_match["home_station"],
    date_time=departure_time,
    max_results=5
)

print(connections)


ConnectionError: HTTPSConnectionPool(host='api.deutschebahn.com', port=443): Max retries exceeded with url: /freeplan/v1/journeys?from=Leipzig%20Hbf&to=Berlin%20Hbf&datetime=2025-12-12T17:30:00&from=Leipzig+Hauptbahnhof&to=Berlin+Hauptbahnhof&datetime=2025-12-12T17%3A30%3A00&results=5 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x114e47da0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [57]:
import requests
import xml.etree.ElementTree as ET

# Konfiguration
EVA_ORIGIN = "8011160"  # Berlin Hbf
DATE = "251110"         # YYMMDD
HOUR = "08"             # Stunde
DESTINATION = "Hamburg Hbf"

HEADERS = {
    "DB-Client-Id": DB_CLIENT_ID,
    "DB-Api-Key": DB_API_KEY,
    "Accept": "application/xml"
}

url = f"https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/plan/{EVA_ORIGIN}/{DATE}/{HOUR}"

response = requests.get(url, headers=HEADERS)

print("Statuscode:", response.status_code)
if response.status_code != 200:
    print("Fehler:", response.text)
else:
    root = ET.fromstring(response.content)
    # Filter nach Ziel
    for departure in root.findall(".//Departure"):
        direction = departure.find("direction").text
        if direction == DESTINATION:
            train = departure.find("train").text
            time = departure.find("time").text
            print(f"Zug {train} nach {direction} ab {time}")


Statuscode: 404
Fehler: 


In [51]:
DB_API_KEY

'0b67148f80c0b3fafbbcbb773dc345d3'